dieses Notebook beinhaltet manuelle Tests zum script:"01_build_inventory.py"

In [25]:
from pathlib import Path
import sys
import re

## def load_config()

In [17]:
def load_config(path_str:str):
    """
    Datei lädt die Konfigurationen aus dem übergebenen Pfad.yaml
    """
    path = Path(path_str)
    if not path.exists():
        print(f"❌ Nicht gefunden unter: {path.resolve()}")
        sys.exit(1)
    else:
        print(f"✅ Config gefunden: {path.resolve()}")

In [3]:
base = Path.cwd()
base = base.parent / "new"
yaml_file = Path("../config/pipeline.yaml")
#load_config(base)
load_config(yaml_file)

✅ Config gefunden: C:\Users\mghan\OneDrive\01.Learn AI\01.BA_Birdmonitoring\project\config\pipeline.yaml


In [18]:
load_config(yaml_file)

✅ Config gefunden: C:\Users\mghan\OneDrive\01.Learn AI\01.BA_Birdmonitoring\project\config\pipeline.yaml


* bei falschem Pfad wird ein Error angegeben -> Test erfolgreich

## laden der Konfigurationen und regex

* Alles Erfolgreich

In [20]:
# Aktueller Ordner vom Notebook (meist .../project/notebooks oder .../project)
cwd = Path.cwd().resolve()

# Falls du im notebooks/ Ordner bist, nimm den Parent als Projekt-Root
project_root = cwd.parent if cwd.name == "notebooks" else cwd

# Projekt-Root ganz nach vorne in sys.path
sys.path.insert(0, str(project_root))

from scripts.utils import load_config


In [ ]:
config_path = "../config/pipeline.yaml"
cfg = load_config(config_path)
print(cfg)

{'paths': {'audio_dir': 'D:/Samsung SSD 850 EVO extern/wmv', 'inventory_csv': 'project/outputs/inventory.csv', 'qc_inventory_csv': 'project/logs/qc_inventory_anomalies.csv', 'sun_events_csv': 'project/outputs/sun_events.csv', 'pipeline_log': 'project/logs/pipeline.log'}, 'scan': {'filename_regex': '^(?P<rec>[^_]+)_(?P<date>\\d{8})_(?P<time>\\d{6})\\.wav$'}, 'session_rules': {'morning_hours': [3, 11], 'evening_hours': [16, 23]}, 'birdnet_week48': {'enabled': True}, 'sun_checks': {'tolerance_min': 10, 'allowed_slot_min': [-120, -60, 0, 60, 120], 'enable_join_sun_events': False}, 'output': {'write_parquet': False}}


In [71]:
config_path = "../config/pipeline.yaml"
cfg = load_config(config_path)

# 2. Hilfsfunktion nutzen. Jetzt haben wir die Konfigurationen geladen
print(f"Lade Konfiguration: {config_path}")

# Pfade aus YAML holen (Relativ zu Projekt-Root oder Absolut)
audio_dir = Path(cfg["paths"]["audio_dir"]) 
output_csv = Path(cfg["paths"]["inventory_csv"])
qc_csv = Path(cfg["paths"]["qc_inventory_csv"])

# Regex aus YAML kompilieren
filename_pattern = re.compile(cfg["scan"]["filename_regex"], re.IGNORECASE)

morning = cfg["session_rules"]["morning_hours"]
evening = cfg["session_rules"]["evening_hours"]

print(f"Scanne Ordner: {audio_dir}")
if not audio_dir.exists():
    sys.exit(f"FEHLER: Audio-Ordner existiert nicht: {audio_dir}")

Lade Konfiguration: ../config/pipeline.yaml
Scanne Ordner: D:\Samsung SSD 850 EVO extern\wmv


In [30]:
print(audio_dir, '\n', output_csv, '\n', evening)
print(filename_pattern)

D:\Samsung SSD 850 EVO extern\wmv 
 project\outputs\inventory.csv 
 [16, 23]
re.compile('^(?P<rec>[^_]+)_(?P<date>\\d{8})_(?P<time>\\d{6})\\.wav$')


## files in ordner finden

In [34]:
len(list(audio_dir.rglob("*.wav")))

427

In [39]:
files = sorted(list(set(audio_dir.rglob("*.wav")) | set(audio_dir.rglob("*.WAV"))))    
print(f"{len(files)} Dateien gefunden.")

427 Dateien gefunden.


In [75]:
files[:100]

[WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250406_181000.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250406_201005.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250407_060001.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250407_080005.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250407_181500.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250407_201505.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250408_055500.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250408_075505.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250408_181500.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250408_201505.WAV'),
 WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250409_055500.WAV'),
 WindowsPath('D:/Samsung SSD 850

In [41]:
# files
# WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250415_202505 (1).WAV'), sollte als fehler ausgegeben werden da hier das re 
# Muster nicht eingehalten wird

Die ursprüngliche Version ist falsch da windows nicht auf groß und kleinschreibung achtet

In [37]:
# Alle WAVs finden (rekursiv mit rglob, falls Unterordner existieren)
files = sorted(list(audio_dir.rglob("*.wav")) + list(audio_dir.rglob("*.WAV")))
print(f"{len(files)} Dateien gefunden.")

854 Dateien gefunden.


## Dateigröße prüfen

* Alles Richtig

In [65]:
p

WindowsPath('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250505_050001.WAV')

In [80]:
# A) Dateigröße prüfen
row = {}
p = files[110]
stat = p.stat()
size_bytes = stat.st_size
row["size_bytes"] = size_bytes
row["filename"] = p.name
row["filepath"] = str(p) # Absoluter Pfad für den Reader später

In [46]:
stat

os.stat_result(st_mode=33206, st_ino=486923061856, st_dev=2122520178, st_nlink=1, st_uid=0, st_gid=0, st_size=1382208864, st_atime=1767038452, st_mtime=1746421200, st_ctime=1767038453)

In [50]:
row['size_bytes']/1000_000_000

1.382208864

In [52]:
row

{'size_bytes': 1382208864,
 'filename': '2453AC0263FBD00C_20250505_050001.WAV',
 'filepath': 'D:\\Samsung SSD 850 EVO extern\\wmv\\2453AC0263FBD00C_20250505_050001.WAV'}

## Regex

In [116]:
p = Path('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250415_202505.WAV')
p2 = Path('D:/Samsung SSD 850 EVO extern/wmv/2453AC0263FBD00C_20250415_202505 (2).WAV')

In [117]:
p.exists(), p2.exists()

(True, True)

In [95]:
row={}
match = filename_pattern.match(p.name)
if not match:
    row["scan_status"] = "bad_filename"
    row["last_error"] = "Regex mismatch"
    

In [96]:
filename_pattern, p.name

(re.compile(r'^(?P<rec>[^_]+)_(?P<date>\d{8})_(?P<time>\d{6})\.wav$',
            re.IGNORECASE|re.UNICODE),
 '2453AC0263FBD00C_20250415_202505.WAV')

In [97]:
row

{}

In [102]:
info = match.groupdict()
row = {}

## Datum extrahieren

* hat funktioniert

In [104]:
from datetime import datetime

In [110]:
# Datum parsen: YYYYMMDD + HHMMSS
dt_str = f"{info['date']}{info['time']}"
start_dt = datetime.strptime(dt_str, "%Y%m%d%H%M%S")
row["recorder_id"] = info["rec"]
row["start_dt"] = start_dt
row["date"] = start_dt.date()
row["month"] = start_dt.month
row["day"] = start_dt.day

In [111]:
print(dt_str)
print(start_dt)
row

20250415202505
2025-04-15 20:25:05


{'recorder_id': '2453AC0263FBD00C',
 'start_dt': datetime.datetime(2025, 4, 15, 20, 25, 5),
 'date': datetime.date(2025, 4, 15),
 'month': 4,
 'day': 15}

## Sounfile wav lesen

In [112]:
row

{'recorder_id': '2453AC0263FBD00C',
 'start_dt': datetime.datetime(2025, 4, 15, 20, 25, 5),
 'date': datetime.date(2025, 4, 15),
 'month': 4,
 'day': 15}

In [122]:
import soundfile as sf
# sf.info liest nur den Header, sehr schnell!
sf_info = sf.info(str(p))

row["duration_s"] = sf_info.duration
row["samplerate"] = sf_info.samplerate
row["channels"] = sf_info.channels
row["format"] = sf_info.format      # z.B. WAV
row["subtype"] = sf_info.subtype    # z.B. PCM_16

In [123]:
sf_info

D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FBD00C_20250415_202505.WAV
samplerate: 96000 Hz
channels: 1
duration: 1:6e+01:55.000 h
format: WAV (Microsoft) [WAV]
subtype: Signed 16 bit PCM [PCM_16]

In [124]:
row

{'recorder_id': '2453AC0263FBD00C',
 'start_dt': datetime.datetime(2025, 4, 15, 20, 25, 5),
 'date': datetime.date(2025, 4, 15),
 'month': 4,
 'day': 15,
 'duration_s': 7195.0,
 'samplerate': 96000,
 'channels': 1,
 'format': 'WAV',
 'subtype': 'PCM_16'}

In [125]:
from datetime import datetime, timedelta

row["end_dt"] = start_dt + timedelta(seconds=sf_info.duration)

In [126]:
row

{'recorder_id': '2453AC0263FBD00C',
 'start_dt': datetime.datetime(2025, 4, 15, 20, 25, 5),
 'date': datetime.date(2025, 4, 15),
 'month': 4,
 'day': 15,
 'duration_s': 7195.0,
 'samplerate': 96000,
 'channels': 1,
 'format': 'WAV',
 'subtype': 'PCM_16',
 'end_dt': datetime.datetime(2025, 4, 15, 22, 25)}

Inventory Tabelle inspizieren

In [127]:
import pandas as pd

In [128]:
df = pd.read_csv('../outputs/inventory.csv')

In [129]:
df

,size_bytes,filename,filepath,is_empty,wav_readable,scan_status,recorder_id,start_dt,date,month,...,duration_s,samplerate,channels,format,subtype,end_dt,birdnet_status,perch_status,updated_at,last_error
0,1382400865,2453AC0263FBD00C_20250406_181000.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-04-06 18:10:00,2025-04-06,4.0,...,7200.000000,96000.0,1.0,WAV,PCM_16,2025-04-06 20:10:00.000000,pending,pending,2026-01-03 18:42:45.796342,NaN
1,1381440865,2453AC0263FBD00C_20250406_201005.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-04-06 20:10:05,2025-04-06,4.0,...,7195.000000,96000.0,1.0,WAV,PCM_16,2025-04-06 22:10:00.000000,pending,pending,2026-01-03 18:42:45.796342,NaN
2,1382208865,2453AC0263FBD00C_20250407_060001.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-04-07 06:00:01,2025-04-07,4.0,...,7199.000000,96000.0,1.0,WAV,PCM_16,2025-04-07 08:00:00.000000,pending,pending,2026-01-03 18:42:45.797364,NaN
3,690240864,2453AC0263FBD00C_20250407_080005.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-04-07 08:00:05,2025-04-07,4.0,...,3595.000000,96000.0,1.0,WAV,PCM_16,2025-04-07 09:00:00.000000,pending,pending,2026-01-03 18:42:45.798385,NaN
4,1382400865,2453AC0263FBD00C_20250407_181500.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-04-07 18:15:00,2025-04-07,4.0,...,7200.000000,96000.0,1.0,WAV,PCM_16,2025-04-07 20:15:00.000000,pending,pending,2026-01-03 18:42:45.798385,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,488,2453AC0263FBD00C_20250904_074505.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-09-04 07:45:05,2025-09-04,9.0,...,0.000000,96000.0,1.0,WAV,PCM_16,2025-09-04 07:45:05.000000,pending,pending,2026-01-03 18:42:46.919845,NaN
423,488,2453AC0263FBD00C_20250904_181500.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-09-04 18:15:00,2025-09-04,9.0,...,0.000000,96000.0,1.0,WAV,PCM_16,2025-09-04 18:15:00.000000,pending,pending,2026-01-03 18:42:46.919845,NaN
424,488,2453AC0263FBD00C_20250904_201505.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-09-04 20:15:05,2025-09-04,9.0,...,0.000000,96000.0,1.0,WAV,PCM_16,2025-09-04 20:15:05.000000,pending,pending,2026-01-03 18:42:46.919845,NaN
425,113836409,2453AC0263FBD00C_20250905_055000.WAV,D:\Samsung SSD 850 EVO extern\wmv\2453AC0263FB...,False,True,scanned,2453AC0263FBD00C,2025-09-05 05:50:00,2025-09-05,9.0,...,592.893458,96000.0,1.0,WAV,PCM_16,2025-09-05 05:59:52.893458,pending,pending,2026-01-03 18:42:46.919845,NaN


# Import der Sonnen-Daten überprüfen

In [32]:
from pathlib import  Path
sun_ref_csv = Path("../outputs/reference_sun.csv")
df_sun = pd.read_csv(sun_ref_csv)

# WICHTIG: Strings zurück in Datetime wandeln
df_sun["date"] = pd.to_datetime(df_sun["date"]).dt.date
df_sun["sunrise_naive"] = pd.to_datetime(df_sun["sunrise_naive"])
df_sun["sunset_naive"] = pd.to_datetime(df_sun["sunset_naive"])

In [33]:
df_sun.head()

,date,sunrise_aware,sunset_aware,sunrise_naive,sunset_naive,noon_naive,dst_active
0,2025-01-01,2025-01-01T08:37:56.825656+01:00,2025-01-01T16:35:21.480696+01:00,2025-01-01 08:37:56.825656,2025-01-01 16:35:21.480696,2025-01-01 12:36:19,False
1,2025-01-02,2025-01-02T08:37:48.675225+01:00,2025-01-02T16:36:26.514750+01:00,2025-01-02 08:37:48.675225,2025-01-02 16:36:26.514750,2025-01-02 12:36:47,False
2,2025-01-03,2025-01-03T08:37:37.205239+01:00,2025-01-03T16:37:34.150550+01:00,2025-01-03 08:37:37.205239,2025-01-03 16:37:34.150550,2025-01-03 12:37:15,False
3,2025-01-04,2025-01-04T08:37:22.432622+01:00,2025-01-04T16:38:44.312211+01:00,2025-01-04 08:37:22.432622,2025-01-04 16:38:44.312211,2025-01-04 12:37:42,False
4,2025-01-05,2025-01-05T08:37:04.377751+01:00,2025-01-05T16:39:56.921743+01:00,2025-01-05 08:37:04.377751,2025-01-05 16:39:56.921743,2025-01-05 12:38:09,False


In [49]:
df_sun['date'][10]

datetime.date(2025, 1, 11)

In [48]:
df_sun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           365 non-null    object        
 1   sunrise_aware  365 non-null    object        
 2   sunset_aware   365 non-null    object        
 3   sunrise_naive  365 non-null    datetime64[ns]
 4   sunset_naive   365 non-null    datetime64[ns]
 5   noon_naive     365 non-null    object        
 6   dst_active     365 non-null    bool          
dtypes: bool(1), datetime64[ns](2), object(4)
memory usage: 17.6+ KB


In [58]:
pp = df_sun[df_sun["date"] == zeit]
pp

,date,sunrise_aware,sunset_aware,sunrise_naive,sunset_naive,noon_naive,dst_active
108,2025-04-19,2025-04-19T06:29:06.923598+02:00,2025-04-19T20:35:52.580545+02:00,2025-04-19 06:29:06.923598,2025-04-19 20:35:52.580545,2025-04-19 13:32:03,True


In [61]:
pp.empty

False

In [ ]:
sr = pp.iloc[0]["sunrise_naive"]
ss = pp.iloc[0]["sunset_naive"]

In [65]:
sr, "\n", ss

(Timestamp('2025-04-19 06:29:06.923598'),
 '\n',
 Timestamp('2025-04-19 20:35:52.580545'))

In [56]:
from datetime import date
zeit = date(2025, 4, 19)

In [71]:
def calculate_slot_with_tolerance(file_start_dt, sun_event_dt, event_name, tolerance_min=15):
    """
    Berechnet den Slot (z.B. sunrise_-120) mit Toleranz.
    Robust gegen Zeitzonen und Datentypen.
    """
    # 1. Sicherheits-Check
    if pd.isna(sun_event_dt) or pd.isna(file_start_dt):
        return None, None

    # 2. Zeitzonen entfernen
    try:
        dt_file = pd.to_datetime(file_start_dt).replace(tzinfo=None)
        dt_sun = pd.to_datetime(sun_event_dt).replace(tzinfo=None)
    except Exception:
        return None, None

    # 3. Mathe (Diff in Minuten)
    delta = dt_file - dt_sun
    diff_minutes = delta.total_seconds() / 60.0
    
    # 4. Welcher vollen Stunde sind wir am nächsten?
    # Wir runden auf Stundenbasis, um die "Slots" (60, 120, 180...) zu finden
    hour_offset = round(diff_minutes / 60.0)
    
    # 5. Abweichungs-Check
    expected_minutes = hour_offset * 60.0
    deviation = abs(diff_minutes - expected_minutes)
    
    # 6. Toleranz-Check
    if deviation <= tolerance_min:
        # ÄNDERUNG: Wir rechnen die Stunden zurück in Minuten für das Label
        # z.B. -2 * 60 = -120
        slot_minutes = int(hour_offset * 60)
        return f"{event_name}_{slot_minutes}", diff_minutes
    else:
        return None, diff_minutes

In [72]:
from datetime import datetime
import pandas as pd
anfang = datetime(2025,4,19,22,40)
sonnenuntergang = datetime(2025,4,19,20,35,52)

In [73]:
anfang

datetime.datetime(2025, 4, 19, 22, 40)

In [74]:
sonnenuntergang

datetime.datetime(2025, 4, 19, 20, 35, 52)

In [75]:
calculate_slot_with_tolerance(anfang,sonnenuntergang,"sonnenuntergg")

('sonnenuntergg_120', 124.13333333333334)

In [70]:
round(100/60)

2

In [78]:
round(122.36,1)

122.4